In [1]:
# Vol Surface Constructor

# 1/ Collect Data from YF
# 2/ Process and clean data
# 3/ Calculate IV
# 4/ Contsruct Vol Surface

In [2]:
# Updates

# Make the code more pythonic (use functions and call them rather than jsut blocks of code)

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from scipy.stats import zscore

In [18]:
# INPUTS
ticker = 'SPY'
outlier_penalty = 3

In [5]:
# Import data and convert into dataframe
def import_data(ticker):
    stock_data = yf.Ticker(ticker)
    expiration_dates = stock_data.options
    option_data = []

    # Loop through all expiration dates and collect the data
    for exp in expiration_dates:
        option_chain = stock_data.option_chain(exp)
        
        calls = option_chain.calls
        puts = option_chain.puts
        
        calls['expiration_date'] = exp
        calls['type'] = 'call'
        puts['expiration_date'] = exp
        puts['type'] = 'put'
        
        option_data.append(calls)
        option_data.append(puts)

    options_df = pd.DataFrame(pd.concat(option_data, ignore_index=True))

    return options_df

options_df = import_data(ticker)
options_df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration_date,type
0,SPY241231C00350000,2024-12-31 14:30:08+00:00,350.0,238.13,233.99,236.82,-4.769989,-1.963767,9.0,14,0.000010,True,REGULAR,USD,2024-12-31,call
1,SPY241231C00355000,2024-12-31 15:44:33+00:00,355.0,233.98,228.98,231.79,-10.840012,-4.427748,12.0,42,0.000010,True,REGULAR,USD,2024-12-31,call
2,SPY241231C00360000,2024-12-19 20:43:08+00:00,360.0,229.00,223.98,225.67,0.000000,0.000000,3.0,0,0.000010,True,REGULAR,USD,2024-12-31,call
3,SPY241231C00365000,2024-12-19 20:46:26+00:00,365.0,223.51,218.90,221.86,0.000000,0.000000,98.0,0,0.000010,True,REGULAR,USD,2024-12-31,call
4,SPY241231C00370000,2024-12-20 17:37:39+00:00,370.0,225.52,213.99,216.87,0.000000,0.000000,8.0,1,0.000010,True,REGULAR,USD,2024-12-31,call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7687,SPY270115P00745000,2024-12-26 19:33:13+00:00,745.0,143.80,157.50,162.46,0.000000,0.000000,1.0,2,0.123086,True,REGULAR,USD,2027-01-15,put
7688,SPY270115P00750000,2024-12-24 17:18:36+00:00,750.0,150.02,162.50,167.38,0.000000,0.000000,NaN,0,0.125024,True,REGULAR,USD,2027-01-15,put
7689,SPY270115P00870000,2024-10-10 13:30:05+00:00,870.0,292.17,269.50,274.29,0.000000,0.000000,NaN,0,0.000010,True,REGULAR,USD,2027-01-15,put
7690,SPY270115P00900000,2024-12-16 14:51:59+00:00,900.0,294.67,312.50,317.38,0.000000,0.000000,1.0,0,0.188363,True,REGULAR,USD,2027-01-15,put


In [6]:
# Data Cleaning

def clean_data(options_df):
    # Convert expiration date to datetime
    options_df["expiration_date"] = pd.to_datetime(options_df["expiration_date"])

    # remove rows with 'null' values
    options_df = options_df[~options_df.isnull().any(axis=1)]

    # drop duplicates
    options_df = options_df.drop_duplicates(subset="contractSymbol")

    # drop columns
    options_df.drop(columns=["lastTradeDate","change","percentChange","volume","inTheMoney","contractSize","currency"],inplace=True, errors="ignore")

    # Normalise
    current_price = yf.Ticker(ticker).history(period="1d").Close.iloc[-1]
    options_df["normalised_price"] = options_df["strike"] / current_price
    options_df = options_df[(options_df["normalised_price"] <= 1.2) & (options_df["normalised_price"] >= 0.8)]

    # time to maturity (+ filtering out long date options)
    options_df["time_to_maturity"] = (options_df.expiration_date - pd.to_datetime("today")).dt.days
    options_df = options_df[(options_df["time_to_maturity"] <= 180) & (options_df["time_to_maturity"] > 0)]

    # Filter out thinly traded options (low open interest)
    options_df = options_df[options_df.openInterest > 1000] # may need to change this -> this would need to change for diff tickers

    return options_df

options_df = clean_data(options_df)
options_df


,contractSymbol,strike,lastPrice,bid,ask,openInterest,impliedVolatility,expiration_date,type,normalised_price,time_to_maturity
524,SPY250102C00580000,580.0,6.11,6.08,6.14,1139,0.000010,2025-01-02,call,0.987890,1
529,SPY250102C00585000,585.0,2.70,2.74,2.76,1384,0.070566,2025-01-02,call,0.996406,1
530,SPY250102C00586000,586.0,2.21,2.22,2.24,1083,0.077463,2025-01-02,call,0.998109,1
531,SPY250102C00587000,587.0,1.76,1.75,1.76,1486,0.080942,2025-01-02,call,0.999813,1
533,SPY250102C00589000,589.0,1.07,1.06,1.07,1500,0.088144,2025-01-02,call,1.003219,1
...,...,...,...,...,...,...,...,...,...,...,...
5134,SPY250630P00568000,568.0,15.36,15.82,15.94,1371,0.149560,2025-06-30,put,0.967451,180
5138,SPY250630P00580000,580.0,17.03,19.10,19.19,2325,0.137414,2025-06-30,put,0.987890,180
5139,SPY250630P00585000,585.0,20.54,20.60,20.76,1478,0.132173,2025-06-30,put,0.996406,180
5140,SPY250630P00590000,590.0,22.20,22.28,22.39,1170,0.126275,2025-06-30,put,1.004922,180


In [19]:
# filter calls and puts

def option_type_filter(options_df, option_type):
    return options_df[options_df['type'] == option_type]


calls_df = option_type_filter(options_df, 'call')
puts_df = option_type_filter(options_df, 'put')

In [ ]:
# Outlier Detection for Call Options

def outlier_detection(df, outlier_penalty):
    X = df[['normalised_price', 'time_to_maturity']].values
    IV = df['impliedVolatility'].values

    # Fit a KNN model to the feature data
    k = 5  
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X)

    # Find the k nearest neighbors for each point
    distances, indices = knn.kneighbors(X)

    # Calculate the average IV of the neighbors
    avg_IV_neighbors = []
    for i in range(len(df)):
        # Get the IVs of the k nearest neighbors
        neighbor_IVs = IV[indices[i]]
        avg_IV_neighbors.append(np.mean(neighbor_IVs))

    # Compute the difference between the IV of each point and its neighbors' average IV
    iv_diff = np.abs(IV - avg_IV_neighbors)

    # Define a threshold for outliers 
    threshold = outlier_penalty * np.std(iv_diff)

    # Flag outliers
    outliers = iv_diff > threshold

    # Add the 'Outlier' column to the DataFrame
    df['Outlier'] = outliers

    df = df[df.Outlier == False]
    return df


outlier_detection(calls_df, outlier_penalty)
outlier_detection(puts_df, outlier_penalty)

C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_33156\1035557657.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Outlier'] = outliers
C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_33156\1035557657.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Outlier'] = outliers


,contractSymbol,strike,lastPrice,bid,ask,openInterest,impliedVolatility,expiration_date,type,normalised_price,time_to_maturity,Outlier
595,SPY250102P00560000,560.0,0.02,0.02,0.03,1561,0.216805,2025-01-02,put,0.953825,1,False
596,SPY250102P00565000,565.0,0.04,0.03,0.04,1596,0.187508,2025-01-02,put,0.962341,1,False
597,SPY250102P00570000,570.0,0.08,0.07,0.08,2094,0.165536,2025-01-02,put,0.970857,1,False
598,SPY250102P00571000,571.0,0.10,0.09,0.10,1486,0.163094,2025-01-02,put,0.972561,1,False
600,SPY250102P00573000,573.0,0.15,0.14,0.15,1665,0.157235,2025-01-02,put,0.975967,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
5134,SPY250630P00568000,568.0,15.36,15.82,15.94,1371,0.149560,2025-06-30,put,0.967451,180,False
5138,SPY250630P00580000,580.0,17.03,19.10,19.19,2325,0.137414,2025-06-30,put,0.987890,180,False
5139,SPY250630P00585000,585.0,20.54,20.60,20.76,1478,0.132173,2025-06-30,put,0.996406,180,False
5140,SPY250630P00590000,590.0,22.20,22.28,22.39,1170,0.126275,2025-06-30,put,1.004922,180,False


In [ ]:
# Outlier Detection for Put Options

puts_df = options_df[options_df['type'] == 'put']

# Extract the relevant features (normalized_price, time_to_maturity, and impliedVolatility)
X = puts_df[['normalised_price', 'time_to_maturity']].values
IV = puts_df['impliedVolatility'].values

# Step 1: Fit a KNN model to the feature data
k = 5  # Number of nearest neighbors to consider
knn = NearestNeighbors(n_neighbors=k)
knn.fit(X)

# Step 2: Find the k nearest neighbors for each point
distances, indices = knn.kneighbors(X)

# Step 3: Calculate the average IV of the neighbors
avg_IV_neighbors = []
for i in range(len(puts_df)):
    # Get the IVs of the k nearest neighbors
    neighbor_IVs = IV[indices[i]]
    avg_IV_neighbors.append(np.mean(neighbor_IVs))

# Step 4: Compute the difference between the IV of each point and its neighbors' average IV
iv_diff = np.abs(IV - avg_IV_neighbors)

# Step 5: Define a threshold for outliers (e.g., 2 standard deviations away from the mean IV difference)
threshold = 3 * np.std(iv_diff)

# Step 6: Flag outliers
outliers = iv_diff > threshold

# Step 7: Add the 'Outlier' column to the DataFrame
puts_df['Outlier'] = outliers

puts_df = puts_df[puts_df.Outlier == False]

In [ ]:

# Plot for calls
fig_calls = px.scatter_3d(
    calls_df,
    x="normalised_price", 
    y="time_to_maturity",   
    z="impliedVolatility",  
    color="impliedVolatility",  
    title="Call Options - Implied Volatility Surface",
)

fig_calls.update_traces(marker=dict(size=3))  # Adjust marker size
fig_calls.show()


# Plot for puts
fig_puts = px.scatter_3d(
    puts_df,
    x="normalised_price",  # x-axis (strike price)
    y="time_to_maturity",   # y-axis (time to maturity)
    z="impliedVolatility",  # z-axis (IV)
    color="impliedVolatility",  # Optional: Color by expiration date
    title="Put Options - Implied Volatility Surface"
)
fig_puts.update_traces(marker=dict(size=3))
fig_puts.show()



In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter

# Assume options_df is your DataFrame with columns: 'strike', 'time_to_maturity', and 'IV'

# Create a grid of strike prices and times to maturity (for interpolation)
strike_grid = np.linspace(calls_df['normalised_price'].min(), calls_df['normalised_price'].max(), 50)
maturity_grid = np.linspace(calls_df['time_to_maturity'].min(), calls_df['time_to_maturity'].max(), 50)

# Create a meshgrid of strike price and time to maturity
X, Y = np.meshgrid(strike_grid, maturity_grid)

# Perform cubic spline interpolation to get implied volatilities for the grid points
Z = griddata(
    (calls_df['normalised_price'], calls_df['time_to_maturity']),
    calls_df['impliedVolatility'],
    (X, Y),
    method='cubic'
)

Z_smoothed = gaussian_filter(Z, sigma=1.5)

# Create the 3D surface plot using plotly.graph_objects
fig = go.Figure(data=[go.Surface(z=Z_smoothed, x=X, y=Y)])

# Add title and labels
fig.update_layout(
    title="Implied Volatility Surface (Cubic Spline)",
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity',
        zaxis_title='Implied Volatility'
    )
)

# Show the plot
fig.show()





In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter

# Scatter Plot for Calls (market data)
fig = go.Figure()

# Plot calls market data
fig.add_trace(go.Scatter3d(
    x=calls_df['normalised_price'],
    y=calls_df['time_to_maturity'],
    z=calls_df['impliedVolatility'],
    mode='markers',
    marker=dict(size=3, color=calls_df['impliedVolatility'], colorscale='Viridis'),
    name="Calls Market IV"
))

# Plot puts market data
fig.add_trace(go.Scatter3d(
    x=puts_df['normalised_price'],
    y=puts_df['time_to_maturity'],
    z=puts_df['impliedVolatility'],
    mode='markers',
    marker=dict(size=3, color=puts_df['impliedVolatility'], colorscale='Cividis'),
    name="Puts Market IV"
))

# Create a grid for interpolation (for the surface plot)
strike_grid = np.linspace(calls_df['normalised_price'].min(), calls_df['normalised_price'].max(), 50)
maturity_grid = np.linspace(calls_df['time_to_maturity'].min(), calls_df['time_to_maturity'].max(), 50)

# Create a meshgrid of strike price and time to maturity
X, Y = np.meshgrid(strike_grid, maturity_grid)

# Perform cubic spline interpolation to get implied volatilities for the grid points
Z = griddata(
    (calls_df['normalised_price'], calls_df['time_to_maturity']),
    calls_df['impliedVolatility'],
    (X, Y),
    method='cubic'
)

# Optionally smooth the Z values with a Gaussian filter
Z_smoothed = gaussian_filter(Z, sigma=1.5)

# Add the smoothed implied volatility surface
fig.add_trace(go.Surface(
    z=Z_smoothed,
    x=X,
    y=Y,
    colorscale='Viridis',
    opacity=0.5,
    name="Smoothed IV Surface"
))

# Add title and labels
fig.update_layout(
    title="Implied Volatility Surface (Calls & Puts) with Interpolated Surface",
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity',
        zaxis_title='Implied Volatility'
    )
)

# Show the combined plot
fig.show()


In [ ]:
####### THE CODE BELOW IS TEST ONLY

# This looks at slicing the Vol surface by time and moneyness

In [ ]:
import plotly.graph_objects as go

# Choose a fixed maturity (time to maturity)
fixed_maturity = 180  # Example: choose 30 days to maturity
# Find the index closest to the fixed maturity
maturity_index = np.abs(maturity_grid - fixed_maturity).argmin()

# Extract the corresponding IV values for the fixed maturity
iv_slice = Z_smoothed[maturity_index, :]

# Extract the strike prices (x-axis) for the slice
strike_slice = X[maturity_index, :]

# Market data for calls at the fixed maturity (you might filter your dataframe accordingly)
calls_at_maturity = calls_df[calls_df['time_to_maturity'] == fixed_maturity]

# Create the 2D plot for the slice
fig_slice_maturity = go.Figure()

# Add a line plot showing the slice for the fixed maturity
fig_slice_maturity.add_trace(go.Scatter(
    x=strike_slice,
    y=iv_slice,
    mode='lines',
    name=f"IV Slice at Maturity = {fixed_maturity} days",
    line=dict(color='blue')
))

# Overlay the scatter points of the market data
fig_slice_maturity.add_trace(go.Scatter(
    x=calls_at_maturity['normalised_price'],
    y=calls_at_maturity['impliedVolatility'],
    mode='markers',
    name="Market Data",
    marker=dict(color='red', size=5)
))

# Add titles and labels
fig_slice_maturity.update_layout(
    title=f"Implied Volatility Slice at Maturity = {fixed_maturity} days",
    xaxis_title="Strike Price (Normalized)",
    yaxis_title="Implied Volatility"
)

# Show the plot
fig_slice_maturity.show()



In [ ]:
# Choose a fixed strike price (moneyness)
fixed_strike = 1.0  # Example: choose normalized strike price of 1 (ATM)
# Find the index closest to the fixed strike
strike_index = np.abs(strike_grid - fixed_strike).argmin()

# Extract the corresponding IV values for the fixed strike
iv_slice_strike = Z_smoothed[:, strike_index]

# Extract the maturities (y-axis) for the slice
maturity_slice = Y[:, strike_index]

# Market data for calls at the fixed strike (filter your dataframe accordingly)
calls_at_strike = calls_df[calls_df['normalised_price'] == fixed_strike]

# Create the 2D plot for the slice
fig_slice_strike = go.Figure()

# Add a line plot showing the slice for the fixed strike
fig_slice_strike.add_trace(go.Scatter(
    x=maturity_slice,
    y=iv_slice_strike,
    mode='lines',
    name=f"IV Slice at Strike = {fixed_strike}",
    line=dict(color='blue')
))

# Overlay the scatter points of the market data
fig_slice_strike.add_trace(go.Scatter(
    x=calls_at_strike['time_to_maturity'],
    y=calls_at_strike['impliedVolatility'],
    mode='markers',
    name="Market Data",
    marker=dict(color='red', size=5)
))

# Add titles and labels
fig_slice_strike.update_layout(
    title=f"Implied Volatility Slice at Strike = {fixed_strike}",
    xaxis_title="Time to Maturity (Days)",
    yaxis_title="Implied Volatility"
)

# Show the plot
fig_slice_strike.show()

